In [3]:
import os

path = 'lastfm-dataset-1K'

if not os.path.exists(path):

  !wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz
  !tar -xvzf lastfm-dataset-1K.tar.gz


In [4]:
!ls

drive  lastfm-dataset-1K  lastfm-dataset-1K.tar.gz  sample_data


In [5]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!ls drive/MyDrive/data

feature_df_with_emb.csv  spotify_features.csv


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import scipy.sparse as sp
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np

from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import datetime
import time 
import math
import os
%config Completer.use_jedi = False

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.


In [8]:
path = "drive/MyDrive/data/feature_df_with_emb.csv"

feature_df_with_emb = pd.read_csv(path)

In [9]:
feature_df_with_emb.head()

,Unnamed: 0,artist,track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genres,popularity,genres_emb,emb_x,emb_y,emb_z
0,0,坂本龍一,Composition 0919 (Live_2009_4_15),0.806,0.413,5.0,-12.883,1.0,0.1560,0.987,0.794,0.1130,0.5290,105.089,330253.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
1,1,坂本龍一,Hibari (Live_2009_4_15),0.604,0.283,9.0,-19.811,0.0,0.0583,0.996,0.916,0.1140,0.2700,137.802,540133.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
2,2,坂本龍一,To Stanford (Live_2009_4_15),0.453,0.163,5.0,-20.922,1.0,0.0519,0.990,0.939,0.1010,0.0703,73.926,216853.0,"['japanese indie folk', 'japanese soundtrack',...",39,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
3,3,坂本龍一,Improvisation (Live_2009_4_15),0.217,0.012,5.0,-36.464,1.0,0.0376,0.991,0.859,0.0866,0.0376,80.714,384707.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
4,4,坂本龍一,Glacier (Live_2009_4_15),0.518,0.213,9.0,-28.790,0.0,0.2090,0.869,0.898,0.1030,0.0920,104.191,580707.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493


In [10]:
# Load the dataset
df = pd.read_table('lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', lineterminator='\n', warn_bad_lines=True, names=['user', 'timestamp', 'artist-id', 'artist', 'song-id', 'song'])
df_profile = pd.read_csv('lastfm-dataset-1K/userid-profile.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=True, skiprows=1, names=['user', 'gender', 'age', 'country', 'signup'])

In [11]:
# Check if some songs share a common ID
grouped = df[['song-id', 'song']].groupby(['song-id']).nunique()

In [12]:
# Get rows with song names containing multiple rows
duplicated = df.loc[df['song-id'].isin(grouped[grouped['song'] > 1].index)]
duplicated = duplicated.drop(columns=['user', 'timestamp', 'artist', 'artist-id'])
containis_extra_rows = duplicated.apply(lambda x: pd.Series({'id': x[0], 'song': x[1], 'flag':'\n' in x[1]}), axis=1)
containis_extra_rows = containis_extra_rows.loc[containis_extra_rows['flag']]

In [13]:
# Iterate over those 5k rows to get the extra rows and correct the song name
for index, row in containis_extra_rows.iterrows():
    row_break = row['song'].split('\n', 1)
    song_name = row_break[0]
    df.loc[df['song-id'] == row['id'], 'song'] = song_name
    
    tsv = StringIO(row_break[1])
    df_extra = pd.read_csv(tsv, sep="\t", warn_bad_lines=True, names=['user', 'timestamp', 'artist-id', 'artist', 'song-id', 'song'])
    df = df.append(df_extra, ignore_index=True)

### Fix names

In [14]:
# Check if some songs share a common ID
grouped = df[['song-id', 'song']].groupby(['song-id']).nunique()

In [15]:
# Check how many ids has more than 1 song name
grouped[grouped['song'] > 1].shape[0]

200

In [16]:
ids_to_be_fixed = grouped[grouped['song'] > 1].index

In [17]:
df = df.set_index('song-id')

In [18]:
map_to_fix = df.groupby(df.index)['song'].head(1)

In [19]:
df.loc[ids_to_be_fixed, 'song'] = df.loc[ids_to_be_fixed].reset_index()['song-id'].map(lambda x: map_to_fix[x])
df = df.reset_index()

### Create matrix

In [20]:
matrix_init = df.groupby(['user', 'artist', 'song']).count()

In [21]:
matrix_init = matrix_init['timestamp']

In [22]:
matrix_init = matrix_init.reset_index()

In [23]:
def rating_scaler(row):
    row_array = np.array(row)
    a, new_range= 1, 4
    min_, max_ = row_array.min(), row_array.max()
    old_range = max_ - min_
    
    
    scaled_row = (new_range * (row_array - min_)) / (old_range + 1e-6)  + a
    return pd.Series(scaled_row)

In [24]:
matrix_init_scaled = matrix_init.groupby('user')['timestamp'].apply(rating_scaler)

In [25]:
matrix_init['timestamp'] = matrix_init_scaled.reset_index()['timestamp']

In [26]:
feature_df_with_emb.head()

,Unnamed: 0,artist,track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genres,popularity,genres_emb,emb_x,emb_y,emb_z
0,0,坂本龍一,Composition 0919 (Live_2009_4_15),0.806,0.413,5.0,-12.883,1.0,0.1560,0.987,0.794,0.1130,0.5290,105.089,330253.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
1,1,坂本龍一,Hibari (Live_2009_4_15),0.604,0.283,9.0,-19.811,0.0,0.0583,0.996,0.916,0.1140,0.2700,137.802,540133.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
2,2,坂本龍一,To Stanford (Live_2009_4_15),0.453,0.163,5.0,-20.922,1.0,0.0519,0.990,0.939,0.1010,0.0703,73.926,216853.0,"['japanese indie folk', 'japanese soundtrack',...",39,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
3,3,坂本龍一,Improvisation (Live_2009_4_15),0.217,0.012,5.0,-36.464,1.0,0.0376,0.991,0.859,0.0866,0.0376,80.714,384707.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493
4,4,坂本龍一,Glacier (Live_2009_4_15),0.518,0.213,9.0,-28.790,0.0,0.2090,0.869,0.898,0.1030,0.0920,104.191,580707.0,"['japanese soundtrack', 'japanese classical', ...",62,[7.3752766 4.8944925 0.94349271],7.375277,4.894493,0.943493


### Merge data

In [27]:
matrix_init = matrix_init.rename({'timestamp':'rating', 'song':'track'}, axis=1)

In [28]:
matrix_extended = matrix_init.merge(feature_df_with_emb, on=['artist', 'track'])

In [29]:
matrix_extended.head()

,user,artist,track,rating,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genres,popularity,genres_emb,emb_x,emb_y,emb_z
0,user_000001,2562,Basin Dub,1.057971,1056,0.644,0.431,11.0,-7.676,1.0,0.206,0.0393,0.3,0.159,0.347,139.998,292769.0,"['classic dubstep', 'future garage', 'bass mus...",19,[5.3110935 6.1038699 3.31327275],5.311094,6.10387,3.313273
1,user_000451,2562,Basin Dub,1.045455,1056,0.644,0.431,11.0,-7.676,1.0,0.206,0.0393,0.3,0.159,0.347,139.998,292769.0,"['classic dubstep', 'future garage', 'bass mus...",19,[5.3110935 6.1038699 3.31327275],5.311094,6.10387,3.313273
2,user_000471,2562,Basin Dub,1.000000,1056,0.644,0.431,11.0,-7.676,1.0,0.206,0.0393,0.3,0.159,0.347,139.998,292769.0,"['classic dubstep', 'future garage', 'bass mus...",19,[5.3110935 6.1038699 3.31327275],5.311094,6.10387,3.313273
3,user_000508,2562,Basin Dub,1.166667,1056,0.644,0.431,11.0,-7.676,1.0,0.206,0.0393,0.3,0.159,0.347,139.998,292769.0,"['classic dubstep', 'future garage', 'bass mus...",19,[5.3110935 6.1038699 3.31327275],5.311094,6.10387,3.313273
4,user_000515,2562,Basin Dub,1.027586,1056,0.644,0.431,11.0,-7.676,1.0,0.206,0.0393,0.3,0.159,0.347,139.998,292769.0,"['classic dubstep', 'future garage', 'bass mus...",19,[5.3110935 6.1038699 3.31327275],5.311094,6.10387,3.313273


### Sample Users

In [30]:
no_users = 200
users_list = matrix_extended.user.unique()
users_sampled = np.random.choice(users_list, replace=False, size=no_users)
matrix_sampled = matrix_extended.copy()
matrix_sampled = matrix_sampled.set_index('user')
matrix_sampled = matrix_sampled.loc[users_sampled]
matrix_sampled = matrix_sampled.reset_index()
matrix_sampled = matrix_sampled.sample(frac=1)

In [31]:
matrix_sampled.head()

,user,artist,track,rating,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genres,popularity,genres_emb,emb_x,emb_y,emb_z
237902,user_000108,The Beatles,We Can Work It Out,1.000000,4640,0.378,0.677,2.0,-8.630,1.0,0.0298,0.0635,0.000000,0.1830,0.648,106.071,135693.0,"['rock', 'british invasion', 'psychedelic rock...",89,[4.9554676 7.5659605 0.6434101],4.955468,7.565960,0.643410
696273,user_000125,Datarock,Nightflight To Uranus,1.027586,108180,0.554,0.782,0.0,-7.085,0.0,0.0699,0.0136,0.071600,0.1050,0.512,133.428,256632.0,"['alternative dance', 'dance-punk', 'bergen in...",40,[5.7524385 7.5104457 2.26829565],5.752439,7.510446,2.268296
42234,user_000205,Shakira,Antologia,1.258065,190887,0.839,0.403,0.0,-11.059,1.0,0.0308,0.5000,0.000002,0.0786,0.575,97.967,254240.0,"['latin pop', 'dance pop', 'pop', 'colombian p...",88,[-1.1042608 6.972612 2.37171375],-1.104261,6.972612,2.371714
134649,user_000799,Shakira,Ojos Así,1.000000,367733,0.717,0.952,6.0,-4.902,1.0,0.0481,0.1500,0.000000,0.1660,0.665,126.993,235733.0,"['latin', 'dance pop', 'colombian pop', 'latin...",88,[-1.1042608 6.972612 2.37171375],-1.104261,6.972612,2.371714
104497,user_000675,Bob Sinclar,Life,1.026490,390407,0.833,0.795,8.0,-7.699,1.0,0.0876,0.0384,0.008140,0.0380,0.895,130.003,342169.0,"['edm', 'dance pop', 'disco house', 'filter ho...",69,[5.641451 5.9498002 3.87226375],5.641451,5.949800,3.872264


In [32]:
matrix_sampled['user_id'] = matrix_sampled['user'].astype('category').cat.codes
matrix_sampled['item_id'] = matrix_sampled['track'].astype('category').cat.codes

In [197]:
X = matrix_sampled[['user_id', 
              'item_id',
              'danceability',
              'energy',
              'speechiness', 
              'acousticness', 
              'instrumentalness', 
              'liveness',
              'valence', 
              'tempo', 
              'popularity', 
              'emb_x', 
              'emb_y',
              'emb_z']]
y = matrix_sampled['rating']
groups = matrix_sampled['user_id']

In [198]:
# Change splitting to be by index using https://stackoverflow.com/questions/53490497/getting-validation-set-from-train-set-by-using-percentage-from-groupby-in-pand
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=groups)

In [199]:
no_users, no_items, no_factors = matrix_sampled['user_id'].nunique(), matrix_sampled['item_id'].nunique(), 20

### Models

In [200]:
def create_shallow_model(no_factors, no_users, no_items):
    # User branch
    user_id = tf.keras.layers.Input(shape=[1], name='user_id')
    user_matrix = tf.keras.layers.Embedding(no_users+1, no_factors, name='user_matrix')(user_id)
    user_vector = tf.keras.layers.Flatten(name='user_vector')(user_matrix)
    # Item branch
    item_id = tf.keras.layers.Input(shape=[1], name='item_id')
    item_matrix = tf.keras.layers.Embedding(no_items+1, no_factors, name='item_matrix')(item_id)
    item_vector = tf.keras.layers.Flatten(name='item_vector')(item_matrix)
    # Dot product 
    vectors_product = tf.keras.layers.dot([user_vector, item_vector], axes=1, normalize=False)
    # Model definition
    model = tf.keras.models.Model(inputs=[user_id, item_id], outputs=[vectors_product], name='shallow_model')
    return model

In [201]:
model = create_shallow_model(no_factors, no_users, no_items)

In [202]:
model.summary()

Model: "shallow_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_matrix (Embedding)         (None, 1, 20)        4020        user_id[0][0]                    
__________________________________________________________________________________________________
item_matrix (Embedding)         (None, 1, 20)        4877080     item_id[0][0]                    
______________________________________________________________________________________

In [203]:
def create_deep_model(no_factors, no_users, no_items):
    # User branch
    user_id = tf.keras.layers.Input(shape=[1], name='user_id')
    user_matrix = tf.keras.layers.Embedding(no_users+1, no_factors, name='user_matrix')(user_id)
    user_vector = tf.keras.layers.Flatten(name='user_vector')(user_matrix)
    # Item branch
    item_id = tf.keras.layers.Input(shape=[1], name='item_id')
    item_matrix = tf.keras.layers.Embedding(no_items+1, no_factors, name='item_matrix')(item_id)
    item_vector = tf.keras.layers.Flatten(name='item_vector')(item_matrix)
    # Concantenation
    vectors_concat = tf.keras.layers.Concatenate()([user_vector, item_vector])
    vectors_concat_dropout = tf.keras.layers.Dropout(0.2)(vectors_concat)
    # Backbone 
    dense_1 = tf.keras.layers.Dense(16,name='fc3')(vectors_concat_dropout)
    dropout_1 = tf.keras.layers.Dropout(0.2,name='d3')(dense_1)
    dense_2 = tf.keras.layers.Dense(8,name='fc4', activation='relu')(dropout_1)
    dense_2_output = tf.keras.layers.Dense(1, activation='relu', name='activation')(dense_2)
    # Model definition
    model = tf.keras.models.Model(inputs=[user_id, item_id], outputs=[dense_2_output], name='deep_model')
    return model

In [204]:
model = create_deep_model(no_factors, no_users, no_items)

### Model training

In [205]:
y_train.head()

288204    1.102564
383279    1.131148
315291    2.400000
359867    1.000000
610089    1.000000
Name: rating, dtype: float64

In [206]:
# Input-output data definition
X_train_arr = [X_train.user_id, X_train.item_id]

# Model creation
model = create_deep_model(no_factors, no_users, no_items)

# Model compiling 
model.compile(loss=tf.keras.losses.MeanSquaredError())

# Model training
model.fit(X_train_arr, y_train, epochs=10, batch_size=2048, shuffle=True)

Epoch 1/10
310/310 [==============================] - 9s 25ms/step - loss: 0.1840
Epoch 2/10
310/310 [==============================] - 8s 25ms/step - loss: 0.1086
Epoch 3/10
310/310 [==============================] - 8s 25ms/step - loss: 0.1045
Epoch 4/10
310/310 [==============================] - 8s 25ms/step - loss: 0.1002
Epoch 5/10
310/310 [==============================] - 8s 25ms/step - loss: 0.0960
Epoch 6/10
310/310 [==============================] - 8s 25ms/step - loss: 0.0928
Epoch 7/10
310/310 [==============================] - 8s 25ms/step - loss: 0.0900
Epoch 8/10
310/310 [==============================] - 8s 25ms/step - loss: 0.0877
Epoch 9/10
310/310 [==============================] - 8s 25ms/step - loss: 0.0858
Epoch 10/10
310/310 [==============================] - 8s 25ms/step - loss: 0.0841


### Prediction

In [207]:
# Predictions in the training set
# X_train = [X_train.user_id, X_train.item_id]
y_train_pred = model.predict(X_train_arr, batch_size=2048)

In [208]:
X_test.head()

,user_id,item_id,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,emb_x,emb_y,emb_z
99280,150,143236,0.602,0.5760,0.0345,0.1820,0.00840,0.5770,0.2130,129.020,58,6.103456,5.685407,3.022125
103073,150,223570,0.431,0.4050,0.0390,0.9440,0.86200,0.3480,0.8880,91.583,22,5.595990,8.315579,1.584192
679335,177,213186,0.127,0.0941,0.0338,0.7240,0.00159,0.0561,0.1360,175.492,42,4.635859,8.002898,0.684818
605104,85,143700,0.472,0.1180,0.0341,0.9870,0.00537,0.1210,0.0688,159.858,54,5.289314,6.018984,2.850651
12616,7,126006,0.195,0.0524,0.0524,0.0107,0.96600,0.0756,0.0356,74.721,48,7.460937,4.877966,0.994511


In [209]:
# Predictions in the test set
X_test_arr = [X_test.user_id, X_test.item_id]
y_test_pred = model.predict(X_test_arr, batch_size=2048)

### Evaluation

In [210]:
from sklearn.metrics import mean_squared_error
print('Train RMSE:', mean_squared_error(y_train.values, y_train_pred, squared=False))
print('Test RMSE:', mean_squared_error(y_test.values, y_test_pred, squared=False))

Train RMSE: 0.2794109985714386
Test RMSE: 0.3367240328038402


In [211]:
def predict_from_latent(model, uid, pids, train_ratings=None):
    user_vector = model.get_layer('user_matrix').get_weights()[0][uid]
    item_vectors = model.get_layer('item_matrix').get_weights()[0][pids]
    scores = (np.dot(user_vector, item_vectors.T))
    return scores

In [212]:
def precision_at_k(model, pred_func, train_ratings, test_ratings, no_users, no_items, k=5):
    pid_array = np.arange(no_items, dtype=np.int32)
    precisions = []
    # For each user
    for user_id, user_test_rating in tqdm(test_ratings.groupby('user_id')):
        # Retrieve already-seen items
        train_pids = train_ratings[train_ratings['user_id'] == user_id]['item_id'].values
        # Retrieve the unseen items
        test_pids = set(user_test_rating['item_id'].values)
        test_set = pd.DataFrame({'user_id': user_id, 'item_id': pid_array})
        test_set = [test_set.user_id, test_set.item_id]
        # Make rating predictions for all items for that user
        predictions = model.predict(test_set, batch_size=2048)
        # predictions = pred_func(model, user_id, pid_array, train_ratings)
        predictions = predictions.reshape(-1)
        # Force a low rating to already-seen items
        predictions[train_pids] = - math.inf
        # Sort the items and het the top k
        top_k = set(np.argsort(-predictions)[:k])
        # Compute precision as per definition
        precisions.append(len(top_k & test_pids) / float(k))
    return precisions

In [213]:
X_train_arr = np.array(X_train_arr)
X_test_arr = np.array(X_test_arr)

In [215]:
train_ratings = pd.DataFrame({'user_id': X_train_arr[0, :], 'item_id':X_train_arr[1, :], 'rating': y_train})
test_ratings = pd.DataFrame({'user_id': X_test_arr[0, :], 'item_id':X_test_arr[1, :], 'rating': y_test})

ValueError: ignored

In [216]:
precisions = precision_at_k(model, predict_from_latent, train_ratings, test_ratings, no_users, no_items, k=10)

100%|██████████| 200/200 [01:08<00:00,  2.91it/s]


In [217]:
np.mean(precisions), np.std(precisions)

(0.0005, 0.007053367989832943)

In [218]:
### EXERCISE CELL ###
def recall_at_k(model, pred_func, train_ratings, test_ratings, no_users, no_items, k=10):
    pid_array = np.arange(no_items, dtype=np.int32)
    recalls = []
    for user_id, user_test_rating in tqdm(test_ratings.groupby('user_id')):
        train_pids = train_ratings[train_ratings['user_id'] == user_id]['item_id'].values
        test_pids = set(user_test_rating['item_id'].values)
        test_set = pd.DataFrame({'user_id': user_id, 'item_id': pid_array})
        test_set = [test_set.user_id, test_set.item_id]
        # Make rating predictions for all items for that user
        predictions = model.predict(test_set, batch_size=2048)
        predictions = predictions.reshape(-1)
        predictions[train_pids] = - math.inf
        top_k = set(np.argsort(-predictions)[:k])
        recalls.append(len(top_k & test_pids) / len(test_pids))
    return recalls

In [219]:
recalls = recall_at_k(model, predict_from_latent, train_ratings, test_ratings, no_users, no_items, k=10)

100%|██████████| 200/200 [01:07<00:00,  2.95it/s]


In [220]:
np.mean(recalls), np.std(recalls)

(4.340277777777778e-06, 6.122715268952207e-05)

In [221]:
### EXERCISE CELL ###
def map_at_k(model, pred_func, train_ratings, test_ratings, no_users, no_items, k=10):
    pid_array = np.arange(no_items, dtype=np.int32)
    maps = []
    for user_id, user_test_rating in tqdm(test_ratings.groupby('user_id')):
        train_pids = train_ratings[train_ratings['user_id'] == user_id]['item_id'].values
        test_pids = set(user_test_rating['item_id'].values)
        test_set = pd.DataFrame({'user_id': user_id, 'item_id': pid_array})
        test_set = [test_set.user_id, test_set.item_id]
        # Make rating predictions for all items for that user
        predictions = model.predict(test_set, batch_size=2048)
        predictions = predictions.reshape(-1)
        predictions[train_pids] = - math.inf
        partial_maps = []
        top_k = list(np.argsort(-predictions)[:k])
        for rank, item_id in enumerate(top_k):
            if item_id in test_pids:
                partial_maps.append(len(set(top_k[:rank+1]) & test_pids) / float(rank+1))
        maps.append(.0 if len(partial_maps) == 0 else np.sum(partial_maps) / float(k))
    return maps

In [222]:
maps = map_at_k(model, predict_from_latent, train_ratings, test_ratings, no_users, no_items, k=10)

100%|██████████| 200/200 [01:07<00:00,  2.96it/s]


In [223]:
np.mean(maps), np.std(maps)

(8.333333333333333e-05, 0.0011755613316388237)